In [47]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.ticker as ticker

In [48]:
def sourcefunc_2D(x,y):
    f = (x**2 + y**2)*np.sin(x*y)
    return f

def boundary_2D(x,y):
    b = np.sin(x*y)
    return b

def sourcefunc_3D(x,y,z):
    f = ((y**2)*(x**2) + (y**2)*(z**2) + (z**2)*(x**2))*np.sin(x*y*z)
    return f

def boundary_3D(x,y,z):
    b = np.sin(x*y*z)
    return b

In [53]:
def create_Afuex_2D(sourcefunc_2D, boundary_2D, p):

    # Here I create the T_h and I_h matrices. These have precisely the same form as in the lecture notes. Some manual
    # stuff is done since we are working without elimination of boundary conditions
    h = 1/(2**p)
    N = 1/h
    N = int(N)
    Th = sp.diags([-1, 2, -1], [-1, 0, 1], shape=(N-1, N-1)).toarray()
    T_h = np.pad(Th,1)
    T_h[0,0] = h**2
    T_h[N,N] = h**2
    Ih = sp.diags([1],[0], shape=(N-1, N-1)).toarray()
    I_h = np.pad(Ih,1)
    # The final A_h matrix is construced here. Because of the h^2 * I_{N+1} identity matrix in the very top left 
    # corner and bottom right corner I have to change four values manually from zero to 1
    A_2D = (1/(h**2))*np.kron(T_h, I_h) + (1/(h**2))*np.kron(I_h, T_h)
    A_2D[0,0] = 1
    A_2D[N,N] = 1
    A_2D[(N+1)**2-N-1,(N+1)**2-N-1] = 1
    A_2D[(N+1)**2-1,(N+1)**2-1] = 1
    
    
    # A meshgrid is created here on which I will evalute the source function. This vector is the right size for
    # the final result, but it includes every boundary value also, as evaluated through f. This is obviously wrong
    # as these boundary values should be evaluated through b, so that has to be adjusted. I therefore immediately 
    # introduce b1 and b_end as vectors which are the boundary values on the bottom and top of the grid, respectively.
    # f is also reshaped here to be a vector, not an array.
    x,y = np.mgrid[0: 1: complex(0, N+1), 0: 1: complex(0, N+1)]
    x = x.transpose()
    y = y.transpose()

    f_2D = sourcefunc_2D(x,y)
    f_2D = np.reshape(f_2D, (N+1)*(N+1))

    x_axis = np.linspace(0, 1, num = N+1)
    b1 = boundary_2D(x_axis, 0)
    b_end = boundary_2D(x_axis, 1)
    
    # In this section I overwrite the parts of the f vector that represent boundary terms and next-to-boundary terms.
    # In the first loop I overwrite the first and last parts of f with b1 and b_end, so that the bottom and top of the 
    # 'grid' are boundary values. In the second loop I overwrite values representing the left and right side of the
    # 'grid'. Of course the bottom and left boundaries are just filled with zeros, as sin(xy) is zero when either x
    # or y is zero. In the third loop I overwrite the entries which represent positions next to the right boundary. In
    # the last loop I overwrite the entries which represent positions right below the top boundary. 


    for i in range(0, N+1):
        f_2D[i] = b1[i]
        f_2D[(N+1)*N + i] = b_end[i]

    for i in range(1,N):
        f_2D[i*(N+1)] = 0
        f_2D[i*(N+1)+ N] = boundary_2D(1, i*h)
    
    for i in range(0,N-1):    
        f_2D[2*N+i*(N+1)] = f_2D[2*N+i*(N+1)] + boundary_2D(1, (i+1)*h)/(h**2)
    
    for i in range(0,N-1):     
        f_2D[(N+1)**2-1-2*N+i] = f_2D[(N+1)**2-1-2*N+i] + b_end[i+1]/(h**2)
        
    u_ex_pre_2D = boundary_2D(x,y)
    u_ex_2D = np.reshape(u_ex_pre_2D, (1, (N+1)*(N+1)))
    
    A_2D = sp.csr_matrix(A_2D)
    f_2D = sp.csr_matrix(f_2D).transpose()
    u_ex_2D = sp.csr_matrix(u_ex_2D).transpose()

    return A_2D , f_2D , u_ex_2D, N

In [54]:
def create_Afuex_3D(A_2D, sourcefunc_3D, boundary_3D, p):
    
    h = 1/(2**p)
    N = 1/h
    N = int(N)
    A = A_2D.torarray()

    for i in range((N+1)**2):
        if A[i,i] == 1:
            A[i,i] = A[i,i]
        else:
            A[i,i] = 1.5*A[i,i]
    
    
    I_b1 = sp.eye(N-1).toarray()      
    I_b2 = np.pad(I_b1,1)     # Now we have a padded identity of size (N+1) x (N+1)
    I_b3 = sp.csr_matrix(I_b2)

    I_c1 = sp.diags([-1, -1], [-1, 1], shape=(N-1, N-1)).toarray()
    I_c2 = np.pad(I_c1, 1)
    I_c3 = sp.csr_matrix(I_c2)

    I_d = np.zeros((N+1,N+1))
    I_d[0,0] = 1
    I_d[N,N] = 1
    I_d2 = sp.csr_matrix(I_d)

    off_diag = sp.kron(I_b3, I_b3)

    A_3D = sp.kron(I_b3, A) + sp.kron(I_c3, (N**2)*off_diag) + sp.kron(I_d, sp.eye((N+1)**2))
    
    for i in range((N+1)**2):
        if A_3D[i,i] == 0:
            A_3D[i,i] = 1
        else:
            A[i,i] = A_3D[i,i]
            
            
            
            
    z,y,x = np.mgrid[0: 1: complex(0, N+1), 0: 1: complex(0, N+1), 0: 1: complex(0, N+1)]

    f_3D = sourcefunc_3D(x,y,z)
    f_3D = np.reshape(f_3D, (N+1)**3)

    b_3D = boundary_3D(x,y,z)
    b_3D = np.reshape(b_3D, (N+1)**3)
    
    
    for i in range(0, (N+1)**2): # Overwriting the first and last layer of the 3D cubic grid
        f_3D[i] = b_3D[i]
        f_3D[((N+1)**2)*N + i] = b_3D[((N+1)**2)*N + i]
    

    for i in range(0, (N+1)**2): # right before the last layer
        f_3D[(N-1)*(N+1)**2 + i] = f_3D[(N-1)*(N+1)**2 + i] + b_3D[N*(N+1)**2 + i]*(N**2)
        
    
    for j in range(1, N):   # for each layer j 
    
        for i in range(0,N-1): # next to boundary points on right side  
            f_3D[2*N+i*(N+1) + j*(N+1)**2] = f_3D[2*N+i*(N+1) + j*(N+1)**2] + b_3D[2*N+i*(N+1) + j*(N+1)**2 + 1]*(N**2)
    
        for i in range(0,N-1): # right below boundary points on top    
            f_3D[(N+1)**2-1-2*N+i + j*(N+1)**2] = f_3D[(N+1)**2-1-2*N+i + j*(N+1)**2] + b_3D[(N+1)**2-1-2*N+i + j*(N+1)**2 + N + 1]*(N**2)
        
        for i in range(0, N+1): # actual boundary points on top and bottom
            f_3D[i + j*(N+1)**2] = b_3D[i + j*(N+1)**2]
            f_3D[(N+1)*N + i + j*(N+1)**2] = b_3D[(N+1)*N + i + j*(N+1)**2]

        for i in range(1,N): # actual boundary points on sides
            f_3D[i*(N+1) + j*(N+1)**2] = 0
            f_3D[i*(N+1) + j*(N+1)**2 + N] = b_3D[i*(N+1)+ j*(N+1)**2 + N]
            
            
        u_ex_3D = b_3D
        A_3D = sp.csr_matrix(A_3D)
        f_3D = sp.csr_matrix(f_3D).transpose()
        u_ex_3D = sp.csr_matrix(u_ex_3D).transpose()
    
    return A_3D, f_3D, b_3D, u_ex_3D

In [51]:
#SSOR
#Same method but with a forward and backwards SOR

import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as LA

def spSORit(A,u,w,f):
    N = u.shape[0]
    s0 = u[0]
    u[0] = ( f[0] - A[0,1:].dot(u[1:]) ) /A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    for i in range(1,N-1): #Do N-2 steps automatically following algorithm
        s = u[i]
        u[i] = (f[i]-A[i,:i].dot(u[:i])-A[i,i+1:].dot(u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-A[-1,:-1].dot(u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    return u

def spSORBWit(A,u,w,f): 
    N = u.shape[0]
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-A[-1,:-1].dot(u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    for i in reversed(range(1,N-1)): #Do N-2 steps automatically following algorithm but reversed
        s = u[i]
        u[i] = (f[i]-A[i,:i].dot(u[:i])-A[i,i+1:].dot(u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    s0 = u[0]
    u[0] = (f[0] - A[0,1:].dot(u[1:]))/A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    return u

#SSOR that runs N iterations
def spSSOR(A,u,w,f,N):
    u0 = u
    for i in range(N):
        u0 = SORit(A,u0,w,f)
        u0 = SORBWit(A,u0,w,f)
    return u0

#Define SSOR with stopping criteria on recidual vector / source vector (r/f)
def spSSORsc(A,u0,w,f,sc):
    u = u0
    r = A.dot(u) - f
    err = np.linalg.norm(r.toarray(),2) / np.linalg.norm(f.toarray(),2)
    it = 0  #To count number of iterations
    residuals = [np.linalg.norm(r.toarray(),2)]
    while err > sc:
        it = it+1
        u = spSORit(A,u,w,f)
        u = spSORBWit(A,u,w,f)
        r = A.dot(u) - f
        residuals.append(np.linalg.norm(r.toarray(),2))
        err = np.linalg.norm(r.toarray(),2) / np.linalg.norm(f.toarray(),2)
        if err > 100:
            print("breaking")
            break
    print("number of iterations :", it)
    return u , np.array(residuals)


In [52]:
import time

error_vec_2D = []
error_vec_3D = []
for i in range(2,8):
    # Comment all this out when doing 3D
    A_2D , f_2D, u_ex_2D, N = create_Afuex_2D(sourcefunc_2D, boundary_2D, i)    
    t = time.time()
    #u_2D = la.spsolve(A_2D_csr, f_2D)
    u_2D = DirSolver(A_2D_csr.toarray(),f_2D)
    print("For p:",i,"time is :",time.time()-t)
    error_2D = np.amax(np.absolute(u_2D-u_ex_2D))
    print("p =", i, ": error_2D =", error_2D)
    error_vec_2D.append(error_2D)


    # Comment all this out when doing 2D
    #A_2D , f_2D, u_ex_2D, N = create_Afuex_2D(sourcefunc_2D, boundary_2D, i)    
    #A_3D, f_3D, b_3D , u_ex_3D = create_Afuex_3D(A_2D, sourcefunc_3D, boundary_3D, i)
    #A_3D_csr = sp.csr_matrix(A_3D)
    #u_3D = la.spsolve(A_3D_csr, f_3D)
    #error_3D = np.amax(np.absolute(u_3D-u_ex_3D))
    #print("p =", i, ": error_3D =", error_3D)
    #error_vec_3D.append(error_3D)

NameError: name 'A_2D_csr' is not defined

In [34]:
A = sp.csr_matrix( [[1,0,1],[0,0,0],[0,2,0]] ) 
v = sp.csr_matrix( [[1],[0],[1]] )


print(A[1,1])
print(A[0,:].dot(v))

0
  (0, 0)	2


In [46]:
A = sp.csr_matrix([[4,1,0],[1,4,1],[0,1,4]])
#v0 = A[0,:]
#v1 = A[1,:]
#d = v0-v1

#print(d.toarray())

M = LUfunc(A)
print(M.toarray())

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
[[4 1 0]
 [0 4 1]
 [0 0 4]]


/Users/bjarkehogdall/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
